In [2]:
pip install firebase-admin

Note: you may need to restart the kernel to use updated packages.


In [4]:
import sys
print(sys.version)


3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]


In [6]:
pip install transformers torch emojis

In [8]:
import requests

API_KEY = '58b4bcc7e574420ca04d721e21e233cd'
url = ('https://newsapi.org/v2/top-headlines?'
       'language=en&'
       'pageSize=100&'
       'apiKey=' + API_KEY)

response = requests.get(url)
articles = response.json().get('articles', [])

In [10]:
news_data = []
for article in articles:
    news_data.append({
        'title': article.get('title', ''),
        'description': article.get('description', ''),
        'content': article.get('content', ''),
        'publishedAt': article.get('publishedAt', ''),
        'source': article.get('source', {}).get('name', ''), 
        'author': article.get('author', ''),
        'url': article.get('url', ''),
        'urlToImage': article.get('urlToImage', '')
    })

print(f"Fetched {len(news_data)} articles")


Fetched 27 articles


In [12]:
import json


print(json.dumps(news_data[0], indent=4))

{
    "title": "My Patient Invited Me To Her Home To Watch Her Die. What I Saw When I Arrived Gave Me Goosebumps. - HuffPost",
    "description": "\u201cShe sat down and paused for a few seconds, and then looked at me earnestly and asked, \u2018Will you be there to hold my hand?\u2019\u201d",
    "content": "There was a sudden feeling of guilt and shame when I realized I should have brought Mrs. MacNeil a farewell gift. Maybe some flowers or a teddy bear. It would have been a thoughtful gesture. After al\u2026 [+10649 chars]",
    "publishedAt": "2025-05-25T12:57:14Z",
    "source": "HuffPost",
    "author": "Herag Hamboyan",
    "url": "https://www.huffpost.com/entry/patient-death-with-dignity-canada_n_681b532be4b0529436a1c55e",
    "urlToImage": "https://img.huffingtonpost.com/asset/681ba9b11b000024003d65d3.jpeg?cache=1kLEVVn3Vo&ops=1200_630"
}


In [14]:
   pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [16]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [18]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


In [20]:
def process_news_data(news_data):
    df = pd.DataFrame(news_data)
    df_clean = df.dropna(subset=['content'])
    
    
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
        
    def clean_text(text):
        if not isinstance(text, str):
            return ""
        tokens = word_tokenize(text.lower())
        filtered = [word for word in tokens if word.isalnum() and word not in stop_words]
        return " ".join(lemmatizer.lemmatize(word) for word in filtered)
    df_clean.loc[:, 'processed_content'] = df_clean['content'].apply(clean_text)

    
    return df_clean

In [22]:
processed_df = process_news_data(news_data)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_17488\3504622434.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.loc[:, 'processed_content'] = df_clean['content'].apply(clean_text)


In [24]:
print(f"Processed {len(processed_df)} of {len(news_data)} news items")
print(processed_df[['title', 'processed_content']])

Processed 26 of 27 news items
                                                title  \
0   My Patient Invited Me To Her Home To Watch Her...   
1   Apple Watch and Apple TV operating systems to ...   
2   Scientists may have discovered the most powerf...   
3   Russia and Ukraine swap hundreds of prisoners,...   
4   'Unbelievable disrespect' - Indiana Fever coac...   
5   3D Is Back. This Time, You Can Ditch the Glass...   
6   Louisiana's Nottoway Plantation burns, prompti...   
7   Inside Ukraine's last maternity ward in a regi...   
8   Russia launches largest overnight air attack y...   
9   At Diddy trial, ex-personal assistants recall ...   
10  What’s open and closed on Memorial Day 2025 - CNN   
11  Gaza: 9 children of doctor couple killed in Is...   
12  Five underdogs who could win the Indy 500 - Th...   
13  SpaceX spacecraft reentering Earth's atmospher...   
14  Horoscope for Sunday, May 25, 2025 - Chicago S...   
15  Oregon man who went viral for quitting job, bu...   
1

In [26]:
from transformers import pipeline
import pandas as pd


sentiment_analyzer = pipeline("sentiment-analysis")


results = sentiment_analyzer(processed_df['processed_content'].tolist())


processed_df["sentiment"] = [res["label"] for res in results]
processed_df["sentiment_score"] = [res["score"] for res in results]


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [28]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

cred = credentials.Certificate("newssen-1ba2b-firebase-adminsdk-fbsvc-0d7e71453f.json")
firebase_admin.initialize_app(cred)



In [30]:
db = firestore.client()
for index, row in processed_df.iterrows():
    data = {
        'title': row['title'],
        'content': row['processed_content'],
        'sentiment': row['sentiment'],
        'score': row['sentiment_score']
    }
    db.collection('news_articles').add(data)

print("Articles uploaded successfully!")

Articles uploaded successfully!
